In [ ]:
import tensorflow as tf


### Consume the TFrecord data
https://www.tensorflow.org/programmers_guide/datasets

In [ ]:
filenames = [ './librivox/guidetomen_%02d_rowland_64kb.mp3' % (i,) for i in [1,2,3]]
filenames

In [ ]:
steps_total, steps_leadin = 1024, 64
mel_bins, spectra_bins = 80, 1025

In [ ]:
def dataset_from_mp3(filenames, stub):
    dataset = tf.data.TFRecordDataset([f.replace('.mp3', '_%s.tfrecords') % stub 
                                       for f in filenames])

    features = {
      "mel":          tf.FixedLenFeature([mel_bins*steps_total], tf.float32),
      "spectra_real": tf.FixedLenFeature([spectra_bins*steps_total], tf.int32),
      "spectra_imag": tf.FixedLenFeature([spectra_bins*steps_total], tf.int32),
    }
    
    def _parse_function(example_proto):
        parsed_features = tf.parse_single_example(example_proto, features)
        spectra = tf.stack( [parsed_features["spectra_real"], parsed_features["spectra_imag"], ] )
        return parsed_features["mel"], spectra

    dataset = dataset.map(_parse_function)

    return dataset